In [1]:
import pandas as pd
import sklearn
import numpy as np

In [2]:
df = pd.read_csv("../Binance/btcusdt.csv")
df

,Unnamed: 0,Open,Close,High,Low
0,0,20762,20752,20762,20752
1,1,20752,20742,20752,20742
2,2,20742,20732,20742,20732
3,3,20732,20722,20732,20722
4,4,20722,20742,20742,20722
...,...,...,...,...,...
1045,1045,21052,21042,21052,21042
1046,1046,21042,21032,21042,21032
1047,1047,21032,21052,21052,21032
1048,1048,21052,21032,21052,21032


In [5]:
# #Check if any zero volumes are available
# indexZeros = df[ df['Volume'] == 0 ].index

# df.drop(indexZeros , inplace=True)
# df.loc[(df["Volume"] == 0 )]
# df.isna().sum()

In [6]:
df.head()

,Unnamed: 0,Open,Close,High,Low
0,0,20762,20752,20762,20752
1,1,20752,20742,20752,20742
2,2,20742,20732,20742,20732
3,3,20732,20722,20732,20722
4,4,20722,20742,20742,20722


In [7]:
import pandas_ta as ta

In [8]:
df['ATR'] = df.ta.atr(length=20)
df['RSI'] = df.ta.rsi()
df['EMA55'] = df.ta.ema(length=55)
df['EMA21'] = df.ta.ema(length=21)

from scipy.stats import linregress
def get_slope(array):
    y = np.array(array)
    x = np.arange(len(y))
    slope, intercept, r_value, p_value, std_err = linregress(x, y)
    return slope

In [9]:
backrollingN = 6

In [10]:
df['slopeRSI'] = df['RSI'].rolling(window=backrollingN).apply(get_slope, raw=True)
df['slopeEMA55'] = df['EMA55'].rolling(window=backrollingN).apply(get_slope, raw=True)
df['slopeEMA21'] = df['EMA21'].rolling(window=backrollingN).apply(get_slope, raw=True)

In [11]:
df = df.dropna()

In [12]:
df

,Unnamed: 0,Open,Close,High,Low,ATR,RSI,EMA55,EMA21,slopeRSI,slopeEMA55,slopeEMA21
59,59,20832,20822,20832,20822,12.480119,42.524371,20820.505172,20849.697689,-2.824616,0.825445,-1.187691
60,60,20822,20812,20822,20812,12.356113,40.044249,20820.201416,20846.270626,-2.709997,0.438822,-1.988810
61,61,20812,20802,20812,20802,12.238307,37.677757,20819.551365,20842.246024,-2.595248,0.066007,-2.717100
62,62,20802,20822,20822,20802,12.626392,44.714774,20819.638817,20840.405476,-1.153629,-0.140432,-2.989571
63,63,20822,20832,20832,20822,12.495072,47.883437,20820.080287,20839.641342,0.515025,-0.145621,-2.743766
...,...,...,...,...,...,...,...,...,...,...,...,...
1045,1045,21052,21042,21052,21042,13.249592,45.446271,21043.783935,21059.453422,-0.003830,0.496859,-0.457677
1046,1046,21042,21032,21042,21032,13.087112,42.993541,21043.363080,21056.957657,-0.929651,0.346461,-0.753732
1047,1047,21032,21052,21052,21032,13.432757,48.930055,21043.671541,21056.506961,-1.260351,0.150414,-1.152743
1048,1048,21052,21032,21052,21032,13.761119,43.995987,21043.254700,21054.279055,-1.354439,-0.069243,-1.593403


In [13]:
#Target flexible way
pipdiff = 500*1e-5 #for TP
SLTPRatio = 2 #pipdiff/Ratio gives SL

def mytarget(barsupfront, df1):
    length = len(df1)
    high = list(df1['High'])
    low = list(df1['Low'])
    close = list(df1['Close'])
    open = list(df1['Open'])
    trendcat = [None] * length
    
    for line in range (0,length-barsupfront-2):
        valueOpenLow = 0
        valueOpenHigh = 0
        for i in range(1,barsupfront+2):
            value1 = open[line+1]-low[line+i]
            value2 = open[line+1]-high[line+i]
            valueOpenLow = max(value1, valueOpenLow)
            valueOpenHigh = min(value2, valueOpenHigh)

            if ( (valueOpenLow >= pipdiff) and (-valueOpenHigh <= (pipdiff/SLTPRatio)) ):
                trendcat[line] = 1 #-1 downtrend
                break
            elif ( (valueOpenLow <= (pipdiff/SLTPRatio)) and (-valueOpenHigh >= pipdiff) ):
                trendcat[line] = 2 # uptrend
                break
            else:
                trendcat[line] = 0 # no clear trend
            
    return trendcat

In [14]:
# mytarget(barsfront to take into account, dataframe)
df['mytarget'] = mytarget(16, df)
df.tail()

C:\Users\hites\AppData\Local\Temp/ipykernel_7588/2438764458.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['mytarget'] = mytarget(16, df)


,Unnamed: 0,Open,Close,High,Low,ATR,RSI,EMA55,EMA21,slopeRSI,slopeEMA55,slopeEMA21,mytarget
1045,1045,21052,21042,21052,21042,13.249592,45.446271,21043.783935,21059.453422,-0.003830,0.496859,-0.457677,NaN
1046,1046,21042,21032,21042,21032,13.087112,42.993541,21043.363080,21056.957657,-0.929651,0.346461,-0.753732,NaN
1047,1047,21032,21052,21052,21032,13.432757,48.930055,21043.671541,21056.506961,-1.260351,0.150414,-1.152743,NaN
1048,1048,21052,21032,21052,21032,13.761119,43.995987,21043.254700,21054.279055,-1.354439,-0.069243,-1.593403,NaN
1049,1049,21032,21052,21052,21032,14.073063,49.482039,21043.567033,21054.071868,0.258681,-0.076975,-1.474522,NaN


In [15]:
df = df.dropna()
attributes = ['ATR', 'RSI', 'EMA55', 'EMA21', 'slopeRSI', 'slopeEMA55', 'slopeEMA21']
x = df[attributes]
y = df['mytarget']
print(x)

            ATR        RSI         EMA55         EMA21  slopeRSI  slopeEMA55  \
59    12.480119  42.524371  20820.505172  20849.697689 -2.824616    0.825445   
60    12.356113  40.044249  20820.201416  20846.270626 -2.709997    0.438822   
61    12.238307  37.677757  20819.551365  20842.246024 -2.595248    0.066007   
62    12.626392  44.714774  20819.638817  20840.405476 -1.153629   -0.140432   
63    12.495072  47.883437  20820.080287  20839.641342  0.515025   -0.145621   
...         ...        ...           ...           ...       ...         ...   
1027  12.157618  64.427661  21025.435441  21051.909499  0.270222    1.854614   
1028  12.049737  66.559925  21028.169889  21056.463181  2.113395    2.125113   
1029  12.447250  58.949277  21030.092393  21058.784710  1.259670    2.304318   
1030  12.324887  55.530319  21031.589094  21059.986100 -0.832723    2.232225   
1031  12.208643  52.265812  21032.675197  21060.169182 -2.397660    1.968829   

      slopeEMA21  
59     -1.187691  
6

In [16]:
df.isna().sum()

Unnamed: 0    0
Open          0
Close         0
High          0
Low           0
ATR           0
RSI           0
EMA55         0
EMA21         0
slopeRSI      0
slopeEMA55    0
slopeEMA21    0
mytarget      0
dtype: int64

In [17]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2, random_state=2)

In [18]:
from sklearn.linear_model import SGDClassifier

In [21]:
model = SGDClassifier()

In [22]:
model.fit(xtrain, ytrain)

y_pred_train = model.predict(xtrain)
y_pred_test = model.predict(xtest)

In [23]:
from sklearn.metrics import accuracy_score
accurancy_train = accuracy_score(ytrain, y_pred_train)
accuracy_tesst = accuracy_score(ytest, y_pred_test)

print("accuracy train %.f%% "% (accurancy_train*100))
print("accuracy test %.f%% "% (accuracy_tesst*100))

accuracy train 50% 
accuracy test 54% 


In [24]:
print(df['mytarget'].value_counts()*100/df['mytarget'].count())


pred_test = np.random.choice([0,1,2], len(y_pred_test))
accuracy_tesst = accuracy_score(ytest, pred_test)
print("accuracy gambler %.2f%%" %(accuracy_tesst*100))

2.0    51.181912
1.0    48.818088
Name: mytarget, dtype: float64
accuracy gambler 32.31%


In [25]:
train_index = int(0.8 * len(x))
xtrain, xtest = x[:train_index], x[train_index:]
ytrain, ytest = y[:train_index], y[train_index:]

In [31]:
model2 = SGDClassifier()
model2.fit(xtrain, ytrain)

y_pred_train = model.predict(xtrain)
y_pred_test = model.predict(xtest)

In [32]:
from sklearn.metrics import accuracy_score
accurancy_train = accuracy_score(ytrain, y_pred_train)
accuracy_tesst = accuracy_score(ytest, y_pred_test)

print("accuracy train %.f%% "% (accurancy_train*100))
print("accuracy test %.f%% "% (accuracy_tesst*100))

accuracy train 49% 
accuracy test 62% 


In [33]:
print(df['mytarget'].value_counts()*100/df['mytarget'].count())


pred_test = np.random.choice([0,1,2], len(y_pred_test))
accuracy_tesst = accuracy_score(ytest, pred_test)
print("accuracy gambler %.2f%%" %(accuracy_tesst*100))

2.0    51.181912
1.0    48.818088
Name: mytarget, dtype: float64
accuracy gambler 32.82%


In [34]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import pandas as pd



# Create the contingency mat
matrix_test = confusion_matrix(ytest, y_pred_test)
matrix_train = confusion_matrix(ytrain, y_pred_train)
classification_report_pred = classification_report(ytest, y_pred_test)

# Convert the matrix to a DataFrame for easy display
matrix_df_test = pd.DataFrame(matrix_test, columns=['pred_Downtrend','pred_Uptrend'], index=['Downtrend','Uptrend'])

matrix_df_train = pd.DataFrame(matrix_train, columns=['pred_Downtrend','pred_Uptrend'], index=['Downtrend','Uptrend'])

print(matrix_df_test)
print("=============================================================================")
print(matrix_df_train)
print("=============================================================================")
print(classification_report_pred)

           pred_Downtrend  pred_Uptrend
Downtrend               0            75
Uptrend                 0           120
           pred_Downtrend  pred_Uptrend
Downtrend               0           400
Uptrend                 0           378
              precision    recall  f1-score   support

         1.0       0.00      0.00      0.00        75
         2.0       0.62      1.00      0.76       120

    accuracy                           0.62       195
   macro avg       0.31      0.50      0.38       195
weighted avg       0.38      0.62      0.47       195



C:\Users\hites\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\hites\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\hites\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
